In [1]:
from __future__ import print_function # use the print() function from Python3
from pylab import *
import librosa             # The librosa library
import librosa.display     # librosa's display module (for plotting features)
import IPython.display     # IPython's display module (for in-line audio)
import matplotlib.pyplot as plt # matplotlib plotting functions
import matplotlib.style as ms   # plotting style
import numpy as np    
import pretty_midi
import random
import soundfile as sf
# sf.write('chase.wav', sec, 4410)

In [8]:
midi_data = pretty_midi.PrettyMIDI('Queen.mid')
# Print an empirical estimate of its global tempo
# print(midi_data.estimate_tempo())
# Compute the relative amount of each semitone across the entire song,
# a proxy for key
# total_velocity = sum(sum(midi_data.get_chroma()))
# print([sum(semitone)/total_velocity for semitone in midi_data.get_chroma()])

instruments = []

midi_data.instruments[1].program = 60

t_num = 0
for ins in midi_data.instruments:
    # print(ins)
    t_num += 1
    instruments.append(ins.program)
    print("Track", t_num, ": ", pretty_midi.program_to_instrument_name(ins.program))

Track 1 :  French Horn
Track 2 :  French Horn
Track 3 :  Acoustic Grand Piano
Track 4 :  Electric Bass (finger)
Track 5 :  String Ensemble 1
Track 6 :  Choir Aahs
Track 7 :  Synth Brass 1
Track 8 :  English Horn
Track 9 :  Distortion Guitar
Track 10 :  Distortion Guitar
Track 11 :  Orchestra Hit
Track 12 :  Acoustic Grand Piano
Track 13 :  Timpani


In [4]:
track_num = 5   # to modify depending on the music
to_gen_prog = [midi_data.instruments[track_num-1].program] 

# to_gen_prog = []

# for instru in to_gen_instru:
#     to_gen_prog.append(pretty_midi.instrument_name_to_program(instru))
# print(pretty_midi.program_to_instrument_name(48))

String Ensemble 1


In [5]:
def cos_wave(A, f, t, sr):
    return [A * cos(2*pi*f*t) for t in arange(0.,t,1./sr)]

def fm(A_carrier, f_carrier, t, sr, A_mod, f_mod):
    return [A_carrier * cos(2 * pi * f_carrier * t + A_mod * cos(2 * pi * f_mod * t)) for t in arange(0., t, 1. / sr)]

def am(wave1, wave2):
    res = []
    for index in range(0, len(wave1)):
        res.append(wave1[index] * wave2[index])
    
    return res

In [6]:
instru_dic = {}
for ins in midi_data.instruments:
    if ins.program in to_gen_prog:
        instru_dic[str(ins.program)] = {}
        for note in ins.notes:
            n_time = round(note.start, 5)
#             n_time = note.start
            if str(n_time) not in instru_dic[str(ins.program)]:
                instru_dic[str(ins.program)][str(n_time)] = []
            instru_dic[str(ins.program)][str(n_time)].append(note)
# print(instru_dic)

# one instrument track in instru_dic
output_dic = {}
prog = 0
count = 0
t_stamp = -1
# GROUP = 4
GAP = 1
# GAP = random.randint(1,3)
temp_test = 20
times = []
# notes_top = pretty_midi.PrettyMIDI()
for key in instru_dic:
    instru_dic = instru_dic[key]
    prog = int(key)
for time in instru_dic:
    if float(time) >= 3:
        times.append(float(time))
times.sort()
print(times)
for time in times:
#     temp_test -= 1
#     if temp_test < 0:
#         break
    time = str(time)
    notes = instru_dic[time]
    if t_stamp == -1 or float(time) - float(t_stamp) >= GAP:
        if t_stamp != -1:
            name_top = str(prog)+'_' + t_stamp
            output_dic[t_stamp].append({'wav_top': name_top+'.wav', 'wav_left': name_top+'_left.wav', 'wav_right': name_top+'_right.wav', 'wav_btm': name_top+'_btm.wav'})
            notes_top.instruments.append(instru_top)
            #top
            wave = notes_top.fluidsynth(fs = 44100, sf2_path = './FluidR3_GM.sf2')
            sf.write( './SoundFiles/' + name_top + '.wav', wave, 44100)
#             print(wave)
            #left
            fm_wave = [10* wave[n] * (0.5+sin(2*pi*480*n)) for n in range(len(wave))]
            sf.write( './SoundFiles/' + name_top + '_left.wav', fm_wave, 44100)
#             print(fm_wave)
            #right
            am_wave = [wave[n] * 10 for n in range(len(wave))]
            sf.write( './SoundFiles/' + name_top + '_right.wav', am_wave, 44100)
            #bottom
            notes_top.instruments[0].program = 60
            wave = notes_top.fluidsynth(fs = 44100, sf2_path = './FluidR3_GM.sf2')
            sf.write( './SoundFiles/' + name_top + '_btm.wav', wave, 44100)
#             print(fm_wave)
#             notes_top.write('./SoundFiles/' + name_top + '.midi')
#             print('midi added')
        t_stamp = time
        output_dic[t_stamp] = []
        notes_top = pretty_midi.PrettyMIDI()
        instru_top = pretty_midi.Instrument(program=prog)
    for note in instru_dic[time]:
        note.start = round(note.start, 5)
        note.start -= float(t_stamp)
        note.end = round(note.end, 5)
        note.end -= float(t_stamp)
        instru_top.notes.append(note)
#         print('note added')
        
#         try:
#             notes_top.write(name_top+'.midi')
#             wave = cello_music.synthesize(fs = 44100)
#             wave = notes_top.fluidsynth(fs = 44100, sf2_path = './FluidR3_GM.sf2')
#             sf.write( './SoundFiles/' + name_top + '.wav', wave, 44100)
#             print(wave)
#             fm_wave = [wave[n] * (0.5+sin(2*pi*480*n)) for n in range(len(wave))]
#             sf.write( './SoundFiles/' + name_top + '_left.wav', fm_wave, 44100)
#             print(fm_wave)
#             fm_wave2 = [wave[n] * 10*sin(2*pi*48*n)+ 100*sin(2*pi*4*n) for n in range(len(wave))]
#             sf.write( './SoundFiles/' + name_top + '_right.wav', fm_wave2, 44100)
# #             print(fm_wave2)
#             am_wave = [wave[n] + sin(2*pi*n) for n in range(len(wave))]
#             sf.write( './SoundFiles/' + name_top + '_btm.wav', am_wave, 44100)
#             print(am_wave)
#         IPython.display.display(IPython.display.Audio(data=wave, rate=44100))
#         except:
#             print('Warning: You did not have fluidsynth correctly installed!')

# print(output_dic)


[15.3846, 16.15383, 16.53844, 18.0769, 18.46152, 18.84613, 19.23075, 19.61536, 20.38459, 21.15382, 24.61536, 25.38459, 25.7692, 26.53843, 27.69228, 28.07689, 28.46151, 28.84612, 29.03843, 29.23074, 29.99997, 30.38458, 30.7692, 31.15382, 31.53843, 32.30766, 32.69227, 33.07689, 33.84612, 34.23073, 34.61535, 35.38458, 35.76919, 36.15381, 37.30765, 37.69227, 38.07688, 38.4615, 39.23073, 39.99996, 40.38458, 40.76919, 41.1538, 41.53842, 42.30765, 42.69226, 43.07688, 45.38457, 46.1538, 76.923, 79.23069, 79.99992, 81.53838, 82.30761, 83.07684, 83.84607, 84.6153, 85.38453, 86.15376, 88.46145, 89.23068, 89.61529, 90.76914, 92.3076, 93.84606, 95.38452, 96.92298, 97.69221, 98.46144, 99.23067, 130.7691, 133.07679, 133.84602, 134.61525, 135.38448, 136.15371, 136.92294, 138.4614, 139.23063, 139.99986, 141.53832, 142.30755, 143.07678, 143.84601, 144.61524, 145.38447, 146.1537, 146.92293, 147.69216, 148.46139, 149.23062, 149.99985, 150.76908, 151.53831, 152.30754, 153.07677, 153.26908, 153.846, 154.615

In [123]:
import json
with open('data.json', 'w') as f:
    json.dump(output_dic, f)

In [ ]:
instru_dic = {}
for ins in midi_data.instruments:
    if ins.program in to_gen_prog:
        instru_dic[str(ins.program)] = {}
        for note in ins.notes:
            n_time = round(note.start, 5)
#             n_time = note.start
            if str(n_time) not in instru_dic[str(ins.program)]:
                instru_dic[str(ins.program)][str(n_time)] = []
            instru_dic[str(ins.program)][str(n_time)].append(note)
# print(instru_dic)

# one instrument track in instru_dic
output_dic = {}
prog = 0
count = 0
t_stamp = 0
GROUP = 4
GAP = 1
# GAP = random.randint(1,3)
temp_test = 28
times = []
# notes_top = pretty_midi.PrettyMIDI()
for key in instru_dic:
    instru_dic = instru_dic[key]
    prog = int(key)
for time in instru_dic:
    if float(time) >= 0:
        times.append(float(time))
times.sort()
print(times)
for time in times:
    temp_test -= 1
    if temp_test < 0:
        break
    time = str(time)
    notes = instru_dic[time]
    if count % GROUP == 0:
        t_stamp = time
        output_dic[t_stamp] = []
        notes_top = pretty_midi.PrettyMIDI()
        instru_top = pretty_midi.Instrument(program=prog)
    for note in instru_dic[time]:
        note.start -= float(t_stamp)
        note.end -= float(t_stamp)
        instru_top.notes.append(note)
        # print('note added')
    if count % GROUP == GROUP - 1:
        name_top = str(prog)+'_' + t_stamp
        output_dic[t_stamp].append({'wav_top': name_top+'.wav', 'wav_left': name_top+'_left.wav', 'wav_right': name_top+'_right.wav', 'wav_btm': name_top+'_btm.wav'})
        notes_top.instruments.append(instru_top)
        try:
#             notes_top.write(name_top+'.midi')
#             wave = cello_music.synthesize(fs = 44100)
            wave = notes_top.fluidsynth(fs = 44100, sf2_path = './FluidR3_GM.sf2')
            sf.write( './SoundFiles/' + name_top + '.wav', wave, 44100)
#             print(wave)
#             fm_wave = [wave[n] * (0.5+sin(2*pi*480*n)) for n in range(len(wave))]
#             sf.write( './SoundFiles/' + name_top + '_left.wav', fm_wave, 44100)
#             print(fm_wave)
#             fm_wave2 = [wave[n] * 10*sin(2*pi*48*n)+ 100*sin(2*pi*4*n) for n in range(len(wave))]
#             sf.write( './SoundFiles/' + name_top + '_right.wav', fm_wave2, 44100)
# #             print(fm_wave2)
#             am_wave = [wave[n] + sin(2*pi*n) for n in range(len(wave))]
#             sf.write( './SoundFiles/' + name_top + '_btm.wav', am_wave, 44100)
#             print(am_wave)
#         IPython.display.display(IPython.display.Audio(data=wave, rate=44100))
        except:
            print('Warning: You did not have fluidsynth correctly installed!')
    count += 1

print(output_dic)